In [5]:
def target():
    print("I am target func")

In [6]:
def deco(func):
    def inner():
        print("Go inner")
    return inner

In [7]:
@deco
def target2():
    print("I am target2 func")

In [8]:
target()

I am target func


In [9]:
target2()

Go inner


In [10]:
# 函数变为了inner 不是 target2
target2

<function __main__.deco.<locals>.inner()>

### 促销案例

> 促销方案一

In [79]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple("Customer", "name fidelity")


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    # 总价
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    # 实际价格
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        return f"Customer:{self.customer} <Order> {self.total()} que: {self.due()}"


class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """返回折扣"""


class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):
    def discount(self, order):
        discount_items = {_.product for _ in order.cart}
        return order.total() * .07 if len(discount_items) >= 10 else 0



In [87]:
zs = Customer("张三",0)
ls = Customer("李四",1100)
cart=[LineItem("🍌",4,.5),LineItem("🍎",10,1.5),LineItem("🍉",5,5),]
print(Order(zs,cart,FidelityPromo()))
print(Order(ls,cart,FidelityPromo()))
print("---------------------------->")
bulk_cart = [LineItem("🍌",30,.5),LineItem("🍎",10,1.5)]
print(Order(zs,bulk_cart,BulkItemPromo()))
print(Order(ls,bulk_cart,BulkItemPromo()))
print("---------------------------->")
long_order = [LineItem(str(item),1,1) for item in range(10)]
print(Order(zs,long_order,LargeOrderPromo()))
print("---------------------------->")

Customer:Customer(name='张三', fidelity=0) <Order> 42.0 que: 42.0
Customer:Customer(name='李四', fidelity=1100) <Order> 42.0 que: 39.9
---------------------------->
Customer:Customer(name='张三', fidelity=0) <Order> 30.0 que: 28.5
Customer:Customer(name='李四', fidelity=1100) <Order> 30.0 que: 28.5
---------------------------->
Customer:Customer(name='张三', fidelity=0) <Order> 10 que: 9.3
---------------------------->


> 促销方案二

In [95]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    # 总价
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    # 实际价格
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        return f"Customer:{self.customer} <Order> {self.total()} que: {self.due()}"


def fidelity_promo(order: Order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order: Order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order: Order):
    discount_items = {_.product for _ in order.cart}
    return order.total() * .07 if len(discount_items) >= 10 else 0

promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order):
    return max([_(order) for _ in promos])


zs = Customer("张三", 0)
ls = Customer("李四", 1100)
cart = [LineItem("🍌", 4, .5), LineItem("🍎", 10, 1.5), LineItem("🍉", 5, 5), ]
print(Order(zs, cart, fidelity_promo))
print(Order(ls, cart, fidelity_promo))
print("---------------------------->")
bulk_cart = [LineItem("🍌", 30, .5), LineItem("🍎", 10, 1.5)]
print(Order(zs, bulk_cart, bulk_item_promo))
print(Order(ls, bulk_cart, bulk_item_promo))
print("---------------------------->")
long_order = [LineItem(str(item), 1, 1) for item in range(10)]
print(Order(zs, long_order, large_order_promo))
print("---------------------------->")

print(Order(zs, long_order, best_promo))
print(Order(zs, cart, best_promo))

Customer:Customer(name='张三', fidelity=0) <Order> 42.0 que: 42.0
Customer:Customer(name='李四', fidelity=1100) <Order> 42.0 que: 39.9
---------------------------->
Customer:Customer(name='张三', fidelity=0) <Order> 30.0 que: 28.5
Customer:Customer(name='李四', fidelity=1100) <Order> 30.0 que: 28.5
---------------------------->
Customer:Customer(name='张三', fidelity=0) <Order> 10 que: 9.3
---------------------------->
Customer:Customer(name='张三', fidelity=0) <Order> 10 que: 9.3
Customer:Customer(name='张三', fidelity=0) <Order> 42.0 que: 42.0


> 促销方案三

In [99]:
promos=[]

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order: Order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order: Order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order_promo(order: Order):
    discount_items = {_.product for _ in order.cart}
    return order.total() * .07 if len(discount_items) >= 10 else 0

def best_promo(order):
    return max(promo(order) for promo in promos)

zs = Customer("张三", 0)
ls = Customer("李四", 1100)
cart = [LineItem("🍌", 4, .5), LineItem("🍎", 10, 1.5), LineItem("🍉", 5, 5), ]
long_order = [LineItem(str(item), 1, 1) for item in range(10)]
print(Order(zs, long_order, best_promo))
print(Order(zs, cart, best_promo))

Customer:Customer(name='张三', fidelity=0) <Order> 10 que: 9.3
Customer:Customer(name='张三', fidelity=0) <Order> 42.0 que: 42.0
